In [1]:
! pip install spacy nltk scikit-learn pandas

In [13]:
import spacy, nltk, re, pandas as pd, numpy as np
from spacy import displacy
from IPython.core.display import display, HTML
from sklearn.neural_network import MLPClassifier
nlp = spacy.load("en_core_web_sm")

In [3]:
sentences=[
  'Can you please book a table for me', 'Can you please reserve table for 2',
  'I want to modify my reservation', 'Can you please make reservation for me',
  'Can you change my reservation', 'Can you cancel my reservation'
          ]
doc = [nlp(sentence.lower().replace("please ", "").replace("i want to ", "").replace("can you ", "")) for sentence in sentences]

lis = []
for sent in doc:
  temp_lis = []
  for token in sent:
    # if token.dep_ == 'ROOT':
      temp_lis.append([str(token.text), str(token.pos_), str(token.dep_), str(token.head.text), str(token.head.pos_), [str(child) for child in token.children]])
  lis.append(temp_lis)

def item_to_dataframe(item):
  item = pd.DataFrame(item)
  item.columns = ['token', 'pos', 'dep', 'head', 'head_pos', 'child']
  return item

df = [item_to_dataframe(item) for item in lis]
df = pd.Series(df)
df.iloc[0]

,token,pos,dep,head,head_pos,child
0,book,NOUN,ROOT,book,NOUN,"[table, for]"
1,a,DET,det,table,NOUN,[]
2,table,NOUN,dobj,book,NOUN,[a]
3,for,ADP,dative,book,NOUN,[me]
4,me,PRON,pobj,for,ADP,[]


In [4]:
df.iloc[1]

,token,pos,dep,head,head_pos,child
0,reserve,NOUN,compound,table,NOUN,[]
1,table,NOUN,ROOT,table,NOUN,"[reserve, for]"
2,for,ADP,prep,table,NOUN,[2]
3,2,NUM,pobj,for,ADP,[]


In [5]:
[display(HTML(displacy.render(sent, style = "dep"))) for sent in doc]

[None, None, None, None, None, None]

In [14]:
required_dependencies = {"dobj", "dative", "compound", "prep"}

def is_nan(x):
  try:
    return np.isnan(x)
  except:
    return x is None

def get_children_of_dep(parsed_series, dep = "ROOT"):
  lis = parsed_series.apply(lambda x: [y for y in x["child"][x["dep"] == dep]])
  # nans = lis.loc[:, 0].apply(lambda x: is_nan(x))
  # lis.loc[:, 0][nans] = lis.loc[:, 1][nans]
  # lis = lis.drop([1], axis = 1)
  # lis = lis.apply(lambda x: [str(x) for tex in x], axis = 1)

  # for sent in parsed_list:
  #   for token in sent:
  #     if token.dep_ == dep:
  #       lis.append([str(children) for children in token.children])
  return [x[0] for x in lis]

required_word_relations = {"pobj"}

# def get_dependency(parent_token, child_token, deps):
#   ret = ""
#   for tokens in sent:
#     if str(tokens.text) == parent_token:
#       print(parent_token)
#       if child_token in [str(child) for child in tokens.children]:
#         ret = tokens.dep_
#   return ret

def get_related_words(root_children, parsed):
  lis = []
  parsed["token"] = parsed["token"].astype(str)
  parsed["head"] = parsed["head"].astype(str)
  root_word = parsed["token"][parsed["dep"] == "ROOT"]
  parsed = parsed[parsed["dep"] != "ROOT"]
  for token in root_children:
    temp_lis = parsed[((parsed["token"] == token) | (parsed["head"] == token)) & (~parsed["head"].isin(root_word)) & (parsed["dep"].isin(required_word_relations))]
    if temp_lis.shape[0] > 0:
      lis.append(temp_lis)
  
  return pd.Series(lis)
  # lis = []
  # for tokens in sent:
  #     if str(tokens.text) == token and tokens.dep_ in required_word_relations:
  #       lis.append([str(child) for child in tokens.children])
  #     elif token == str(tokens.head.text):
  #       dep_ = get_dependency(token, str(tokens.text), sent)
  #       print(dep_)
  #       # if dep_ in required_word_relations:
  #       lis.append([tokens.text])
  # return lis

# def get_relations_to_word(parsed_list, words_list):
#   lis = []
#   for i, word in enumerate(words_list):
#     sent = parsed_list[i]
#     temp_lis = []
#     for token in word:
#       temp_lis.append([get_relation(token, sent)])
#     lis.append(temp_lis)
  
#   return lis

In [7]:
root_children = get_children_of_dep(df, "ROOT")
root_children

[['table', 'for'],
 ['reserve', 'for'],
 ['reservation'],
 ['reservation'],
 ['reservation'],
 ['reservation']]

In [15]:
df1 = pd.DataFrame({"parsed": df, "root_children": root_children})

In [9]:
df1["root_children"].iloc[0]

['table', 'for']

In [10]:
df2 = df1.apply(lambda x: get_related_words(x["root_children"], x["parsed"]), axis = 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [11]:
[df2.iloc[i][0] for i in range(df2.shape[0])]

[  token   pos   dep head head_pos child
 4    me  PRON  pobj  for      ADP    [],
   token  pos   dep head head_pos child
 3     2  NUM  pobj  for      ADP    [],
 nan,
 nan,
 nan,
 nan]

In [22]:
results = df1
results.columns= ['parsed_sentence','Intent']
results

,parsed_sentence,Intent
0,token pos dep head head_pos ...,"[table, for]"
1,token pos dep head head_pos ...,"[reserve, for]"
2,token pos dep head head_p...,[reservation]
3,token pos dep head head_p...,[reservation]
4,token pos dep head head_p...,[reservation]
5,token pos dep head head_p...,[reservation]


In [23]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(20,10), random_state=1)

#clf.fit(X, y)

#We can train a model once we have a labelled dataset